In [5]:
# Re-define the entire class structure along with the required imports and functions

import numpy as np
import pandas as pd
from functools import reduce
import warnings
from statsmodels.tsa.vector_ar import vecm
from statsmodels.tsa.vector_ar.vecm import VECM, select_order , select_coint_rank
from analysis_metrics.hasbrouck import InfoShares, ResultsWarper, compshares, ishares, mishares
norm = np.linalg.norm

In [6]:
# Generate synthetic data for testing
#Synthetic data case number 1: RANDOM DATA
np.random.seed(42)
data = pd.DataFrame(np.random.randn(100, 2), columns=['Variable1', 'Variable2'])

# Initialize the InfoShares class with synthetic data
info_shares_model = InfoShares(data, k_ar_diff=1)

# Fit the model
results = info_shares_model.fit()

# Display results
print(results.infoShares)

[[0.75694353 0.24305647]]


In [7]:
#Synthetic data case number 2: Clear lider.
# Generate synthetic data for the second case
np.random.seed(42)
n = 1000
time_series_1 = np.cumsum(np.random.uniform(-1, 1, n))

# Generate random cumulative sums for the first 100 values
initial_cumsum = np.cumsum(np.random.uniform(-1, 1, 100))

# Shift the time series 100 to the future
time_series_2 = np.empty(n)
time_series_2[:100] = np.nan  # Initialize the first 100 values as NaN
time_series_2[100:] = time_series_1[:-100]  # Shift the time series

# Replace NaN values with the generated random cumulative sums
time_series_2[:100] = initial_cumsum

# Create the DataFrame
data_lagged = pd.DataFrame({'Variable1': time_series_1, 'Variable2': time_series_2})

rank_test = select_coint_rank(data_lagged, 0, 1, method="trace",
                              signif=0.1)
#We print the rank test
print(rank_test)
print(rank_test.rank)

# Initialize the InfoShares class with the lagged data
info_shares_model_lagged = InfoShares(data_lagged, k_ar_diff=1)

# Fit the model and compute the metric
results_lagged = info_shares_model_lagged.fit()

# Display the results
print(results_lagged.infoShares)

Johansen cointegration test using trace test statistic with 10% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   2          11.11          13.43
-------------------------------------
0
[[0.98518411 0.01481589]]


In [8]:
#Synthetic data case number 3: Clear lider, but on position 2.
# Generate synthetic data for the third case (reversing the order of variables)
data_lagged_reversed = pd.DataFrame({'Variable1': time_series_2, 'Variable2': time_series_1})

# Initialize the InfoShares class with the reversed lagged data
info_shares_model_lagged_reversed = InfoShares(data_lagged_reversed, k_ar_diff=1)

# Fit the model and compute the metric
results_lagged_reversed = info_shares_model_lagged_reversed.fit()

# Display the results
print(results_lagged_reversed.infoShares)

[[0.01405019 0.98594981]]


In [9]:

#Synthetic data case number 4: Explicitly cointegrated data.
# Generate synthetic data for the cointegrated case
np.random.seed(42)
n = 1000
X_t = np.cumsum(np.random.randn(n))  # Random walk for the first series
alpha = 1.5
epsilon_t = np.random.normal(0, 0.1, n)
Y_t = alpha * X_t + epsilon_t  # Cointegrated series

data_cointegrated = pd.DataFrame({'Variable1': X_t, 'Variable2': Y_t})

# Initialize the InfoShares class with the cointegrated data
info_shares_model_cointegrated = InfoShares(data_cointegrated, k_ar_diff=1)

# Fit the model and compute the metric
results_cointegrated = info_shares_model_cointegrated.fit()

# Display the results
print(results_cointegrated.infoShares)

[[0.99885796 0.00114204]]


In [10]:
#Synthetic data case number 5: Explicitly cointegrated data, lider on position 2.
# Generate synthetic data for the cointegrated case with inverted series
data_cointegrated_inverted = pd.DataFrame({'Variable1': Y_t, 'Variable2': X_t})

# Initialize the InfoShares class with the inverted cointegrated data
info_shares_model_cointegrated_inverted = InfoShares(data_cointegrated_inverted, k_ar_diff=1)

# Fit the model and compute the metric
results_cointegrated_inverted = info_shares_model_cointegrated_inverted.fit()

# Display the results
print(results_cointegrated_inverted.infoShares)

[[0.9896988 0.0103012]]
